In [1]:
import pandas as pd

pd.set_option("display.max_columns", None)

In [2]:
table_path = "../raw_docs/done_EMBA-35. Exit Survey.xlsx"
raw_data = pd.read_excel(table_path, skiprows=[0, 2, 3], sheet_name="Raw data")
processed_data = pd.read_excel(table_path, skiprows=[0, 2, 3], sheet_name="edited")

In [3]:
processed_data.head(3)

,Unnamed: 0,S0 User Agent,S1 Platform,S2 Browser,S3 Device,S4 IP-Address,S5 Source,S6 NumberID,S7 Respondent ID,S8 Link to completed questionnaire,S9 Language,S12 Completed,S13 Disqualified,S14 Start,S15 Completed Date,S16 Duration,S17 Query String,PS0 Start,"PD0 Duration , sec","Q2.1 Удовлетворенность приобретенными на программе знаниями, умениями, навыками",Q2.2 Профессорско-преподавательский состав,Q2.3 Административная поддержка программы,Q2.4 Общее впечатление от программы,Unnamed: 23,Q3.1 Я демонстрирую экспертный уровень знания основных бизнес-дисциплин,Q3.2 Я научился/научилась анализировать и работать с данными для принятия стратегических решений на уровне компании,Q3.3 Я научился/научилась определять стратегии для устойчивого развития и создания долгосрочной ценности компании,Q3.4 Я демонстрирую навыки интеграционного лидерства: мотивировать команды и целые организации на достижение важных стратегических задач.,Q3.5 Я применяю навыки эффективной коммуникации и командного взаимодействия для повышения эффективности,"Q3.6 Я научился/научилась структурировать и коммуницировать стратегические решения, задачи","Q3.7 Я научился/научилась оценивать глобальный контекст, учитывать влияние и потенциальные возможности прорывных технологий в индустриях, бизнесе и организациях","Q3.8 Я научился/научилась формулировать и внедрять инициативы в области ERS в организациях ( Ethics (этика), Responsibility (ответственность) and Sustainability (устойчивое развитие))","Q3.9 Я развил/развила креативность и навыки инновационного мышления, научился/научилась использовать новаторские подходы к решению сложных бизнес-кейсов и задач","Q3.10 Я развил/развила предпринимательское мышление и выстраиваю корпоративную культуру, которая поощряет эксперименты и творческий подход в организации","Q4.1 Каковы, на ваш взгляд, сильные и слабые стороны программы? - Сильные стороны","Q4.2 Каковы, на ваш взгляд, сильные и слабые стороны программы? - Слабые стороны",Q5.1 Логичность и согласованность содержания программы,Q5.2 Баланс между теоретическими и практическими знаниями,Q5.3 Применимость полученных знаний,Q5.4 Актуальность полученных знаний,Q5.5 Баланс между глобальными и локальными/региональными примерами,Q5.6 Достаточность включения проектной работы в учебные модули,Q5.7 Качество выступлений приглашенных спикеров,"Q6.1 🔴 В поле ниже напишите имена пяти профессоров , чьи курсы вам запомнились больше всего - Профессор 1","Q6.2 🔴 В поле ниже напишите имена пяти профессоров , чьи курсы вам запомнились больше всего - Профессор 2","Q6.3 🔴 В поле ниже напишите имена пяти профессоров , чьи курсы вам запомнились больше всего - Профессор 3","Q6.4 🔴 В поле ниже напишите имена пяти профессоров , чьи курсы вам запомнились больше всего - Профессор 4","Q6.5 🔴 В поле ниже напишите имена пяти профессоров , чьи курсы вам запомнились больше всего - Профессор 5",Unnamed: 48,Unnamed: 49,"Q7.1 Качество кейсов, представленных на модулях",Q7.2 Возможность применения полученных на программе знаний,Q7.3 Опыт групповой работы,Q7.4 Подбор локаций для модулей,Unnamed: 54,Q8.1 Отклик на потребности студентов во время программы,Q8.2 Организация образовательного процесса вашей группы,Unnamed: 57,Q9.1 С точки зрения поддержи и взаимопомощи,"Q9.2 С точки зрения опыта и знаний, которые привнесли ваши одногруппники в процессе обучения",Q9.3 С точки зрения разнообразия представленных в группе индустрий,Q9.4 С точки зрения приобретения полезных деловых контактов,Q10 Оставьте ваши пожелания и дополнительные комментарии по программе,Q11.1 В качестве приглашенного спикера,Q11.2 В качестве ментора на проектной работе,Q11.3 Участие в мероприятиях для выпускников программы,Q11.4 Участие в адмиссии Degree,Q11.5 Другое,"Q11.1.O В качестве приглашенного спикера - брендинг, корпфин",Q11.5.O Другое - протагонист кейса,Unnamed: 70,Q12.1 - 🔴 Готовы ли вы порекомендовать программу своим друзьям/коллегам?,Unnamed: 72,Q13.1 Топ-менеджер / руководитель высшего звена,Q13.2 Госслужащий,Q13.3 Начинающий предп

#### Converting table to a desired structure

In [4]:
service_cols = [col for col in raw_data.columns if col.startswith("S")]
unnamed_cols = [col for col in raw_data.columns if col.startswith("Unnamed:")]
cols_to_drop = ["PS0 Start", "PD0 Duration , sec"] + service_cols + unnamed_cols

In [5]:
raw_data = raw_data.drop(columns=cols_to_drop)
processed_data = processed_data.drop(columns=cols_to_drop)

In [6]:
# drop all rows where participant name is None
name_col = "Q16  - 🔴 Укажите  фамилию и имя"
raw_data = raw_data.dropna(subset=[name_col], ignore_index=True)

In [7]:
# replace professors names with their standard form (block Q6)
q6_cols = [col for col in raw_data.columns if col.startswith("Q6")]
raw_data[q6_cols] = processed_data[q6_cols]

#### Filling Nans

In [8]:
raw_data.isna().sum()

Q2.1 Удовлетворенность приобретенными на программе знаниями, умениями, навыками                                                                                                                    0
Q2.2 Профессорско-преподавательский состав                                                                                                                                                         0
Q2.3 Административная поддержка программы                                                                                                                                                          0
Q2.4 Общее впечатление от программы                                                                                                                                                                0
Q3.1 Я демонстрирую экспертный уровень знания основных бизнес-дисциплин                                                                                                                            0
Q3.2 Я научился

In [9]:
raw_data[q6_cols] = raw_data[q6_cols].fillna(
    0
)  # if a student doesnt remember any professors

age_col = "Q15 🔴  Укажите ваш  возраст"
raw_data[age_col] = raw_data[age_col].fillna(
    raw_data[age_col].median()
)  # if the age was not provided fill with mode

score_cols = [
    col
    for col in raw_data.columns
    if col.startswith("Q7") or col.startswith("Q8") or col.startswith("Q9")
]
for col in score_cols:
    raw_data[col] = raw_data[col].fillna(
        int(raw_data[col].mean())
    )  # if a value in scoring col is missing, replace by the mean

free_answer_cols = [
    "Q4.1 Каковы, на ваш взгляд,  сильные и слабые стороны программы? - Сильные стороны",
    "Q4.2 Каковы, на ваш взгляд,  сильные и слабые стороны программы? - Слабые стороны",
    "Q10 Оставьте ваши  пожелания и дополнительные комментарии  по программе",
    "Q10 Оставьте ваши  пожелания и дополнительные комментарии  по программе",
    "Q11.1.O В качестве приглашенного спикера - брендинг, корпфин",
    "Q11.5.O Другое - протагонист кейса",
    "Q13.6.O Другое - Other",
    "Q14.20.O Другое - Other",
]
for col in free_answer_cols:
    raw_data[col] = raw_data[col].fillna(
        "No comments"
    )  # set to "No comments" if the answer wasnt provided in cols with free answer

raw_data.isna().any().any()

np.False_

#### Save the results

In [10]:
raw_data.head(3)

,"Q2.1 Удовлетворенность приобретенными на программе знаниями, умениями, навыками",Q2.2 Профессорско-преподавательский состав,Q2.3 Административная поддержка программы,Q2.4 Общее впечатление от программы,Q3.1 Я демонстрирую экспертный уровень знания основных бизнес-дисциплин,Q3.2 Я научился/научилась анализировать и работать с данными для принятия стратегических решений на уровне компании,Q3.3 Я научился/научилась определять стратегии для устойчивого развития и создания долгосрочной ценности компании,Q3.4 Я демонстрирую навыки интеграционного лидерства: мотивировать команды и целые организации на достижение важных стратегических задач.,Q3.5 Я применяю навыки эффективной коммуникации и командного взаимодействия для повышения эффективности,"Q3.6 Я научился/научилась структурировать и коммуницировать стратегические решения, задачи","Q3.7 Я научился/научилась оценивать глобальный контекст, учитывать влияние и потенциальные возможности прорывных технологий в индустриях, бизнесе и организациях","Q3.8 Я научился/научилась формулировать и внедрять инициативы в области ERS в организациях ( Ethics (этика), Responsibility (ответственность) and Sustainability (устойчивое развитие))","Q3.9 Я развил/развила креативность и навыки инновационного мышления, научился/научилась использовать новаторские подходы к решению сложных бизнес-кейсов и задач","Q3.10 Я развил/развила предпринимательское мышление и выстраиваю корпоративную культуру, которая поощряет эксперименты и творческий подход в организации","Q4.1 Каковы, на ваш взгляд, сильные и слабые стороны программы? - Сильные стороны","Q4.2 Каковы, на ваш взгляд, сильные и слабые стороны программы? - Слабые стороны",Q5.1 Логичность и согласованность содержания программы,Q5.2 Баланс между теоретическими и практическими знаниями,Q5.3 Применимость полученных знаний,Q5.4 Актуальность полученных знаний,Q5.5 Баланс между глобальными и локальными/региональными примерами,Q5.6 Достаточность включения проектной работы в учебные модули,Q5.7 Качество выступлений приглашенных спикеров,"Q6.1 🔴 В поле ниже напишите имена пяти профессоров , чьи курсы вам запомнились больше всего - Профессор 1","Q6.2 🔴 В поле ниже напишите имена пяти профессоров , чьи курсы вам запомнились больше всего - Профессор 2","Q6.3 🔴 В поле ниже напишите имена пяти профессоров , чьи курсы вам запомнились больше всего - Профессор 3","Q6.4 🔴 В поле ниже напишите имена пяти профессоров , чьи курсы вам запомнились больше всего - Профессор 4","Q6.5 🔴 В поле ниже напишите имена пяти профессоров , чьи курсы вам запомнились больше всего - Профессор 5","Q7.1 Качество кейсов, представленных на модулях",Q7.2 Возможность применения полученных на программе знаний,Q7.3 Опыт групповой работы,Q7.4 Подбор локаций для модулей,Q8.1 Отклик на потребности студентов во время программы,Q8.2 Организация образовательного процесса вашей группы,Q9.1 С точки зрения поддержи и взаимопомощи,"Q9.2 С точки зрения опыта и знаний, которые привнесли ваши одногруппники в процессе обучения",Q9.3 С точки зрения разнообразия представленных в группе индустрий,Q9.4 С точки зрения приобретения полезных деловых контактов,Q10 Оставьте ваши пожелания и дополнительные комментарии по программе,Q11.1 В качестве приглашенного спикера,Q11.2 В качестве ментора на проектной работе,Q11.3 Участие в мероприятиях для выпускников программы,Q11.4 Участие в адмиссии Degree,Q11.5 Другое,"Q11.1.O В качестве приглашенного спикера - брендинг, корпфин",Q11.5.O Другое - протагонист кейса,Q12.1 - 🔴 Готовы ли вы порекомендовать программу своим друзьям/коллегам?,Q13.1 Топ-менеджер / руководитель высшего звена,Q13.2 Госслужащий,Q13.3 Начинающий предприниматель с проектом или идеей,Q13.4 Собственник / владелец бизнеса,Q13.5 Менеджер среднего звена / руководитель направления /департамента,Q13.6 Другое,Q13.6.O Другое - Other,Q14 🔴 Укажите в какой отрасли вы ведете деятельность,Q14.20.O Другое - Other,Q15 🔴 Укажите ваш возраст,Q16 - 🔴 Укажите фамилию и имя
0,8.0,8.0,10.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,6.0,8.0

In [11]:
raw_data.to_excel("../artifacts/processed.xlsx", index=False)